# Colab NAIP Canopy Height Model Generator

Generate vegetation and structure height maps using NAIP aerial imagery. Results are saved to your Google Drive. This notebook is designed to be used in Google Colab.

### **Output**
The output is a full NAIP DOQQ (Digital Ortho Quarter Quadrangle) centered on your location of interest. 
*   **Area:** Approximately 4 miles by 4 miles (3.75' latitude x 3.75' longitude).
*   **Format:** UINT16 (unsigned 16-bit integer).
*   **Units:** Centimeters (cm). The data is scaled by 100. Divide pixel values by 100 to obtain height in meters.

### **Requirements**
1. **Google Account:** Access to Google Drive is assumed.
2. **Google Earth Engine Project:** You need an active Cloud Project in Earth Engine.

### **Setup (Crucial Step)**

To run this quickly (~2-6 mins instead of > 2 hours), you must enable a GPU:
1. Click **Connect** in the top menu.
2. Type **Change runtime type** and hit enter.
3. Choose **T4 GPU** and save. 


## 1. Configure
Update the settings below:
*   **Location:** Enter the Latitude (`LAT`) and Longitude (`LON`) for your center point.
*   **Year:** Set the `YEAR` for the imagery.
*   **Project ID:** Replace `'your-gee-project-id'` with your actual Google Earth Engine Project ID.

In [ ]:
# === CONFIGURATION ===

# Inputs
LAT = 46.89  # Latitude of interest in Decimal Degrees
LON = -113.45 # Longitude of interest in Decimal Degrees
YEAR = 2009 # Desired Year

# GEE Project ID (must have access to the assets)
PROJECT_ID = 'your-gee-project-id'

# Output Folder Name in Google Drive
DRIVE_FOLDER_NAME = 'NAIP_CHM_Output'

# Model Config (do not update)
MODEL_PATH = 'model/model_20251016.pt'
CONFIG_PATH = 'configs/config.yaml'

## 2. Setup Environment
Clone the repository and install dependencies.

In [ ]:
# Clone repository
!git clone https://github.com/smorf-ntsg/naip-chm.git
%cd naip-chm

# Install dependencies
!pip install rio-cogeo

## 3. Authenticate and Initialize
Authenticate with Google Earth Engine and mount Google Drive for output.

**Note:** Google will ask for permission to access your Google credentials and Google Drive. You must approve these requests to proceed.

In [ ]:
import ee
import geemap
from google.colab import drive
import os
from pathlib import Path

# Authenticate GEE and Initialize
ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

# Mount Google Drive
drive.mount('/content/drive')

# Construct Full Output Path
full_output_path = Path('/content/drive/MyDrive') / DRIVE_FOLDER_NAME
os.makedirs(full_output_path, exist_ok=True)
print(f"Output directory set to: {full_output_path}")

## 4. Check Imagery Availability
Verify that 4-band NAIP imagery exists for your selected location and year.

In [ ]:
# Define point of interest
point = ee.Geometry.Point([LON, LAT])

# Load NAIP collection and filter by location
naip_col = ee.ImageCollection('USDA/NAIP/DOQQ').filterBounds(point)

# Function to add band count and year
def add_metadata(img):
    bcount = img.bandNames().length()
    year = img.date().get('year')
    return img.set({'band_count': bcount, 'year': year})

# Filter for 4-band imagery and get years
years = naip_col.map(add_metadata).filter(ee.Filter.eq('band_count', 4)).sort('year').aggregate_array('year').distinct()

# Get the list client-side
available_years = years.getInfo()

print(f"Available years with 4-band imagery: {available_years}")

if YEAR in available_years:
    print(f"SUCCESS: {YEAR} is available at ({LAT}, {LON}).")
    print("Proceed to next step.")
else:
    print(f"FAILURE: {YEAR} is NOT available with 4 bands at ({LAT}, {LON}).")
    print(f"Please update the 'YEAR' variable in the configuration cell to one of: {available_years}")

## 5. Run Inference
Initialize the streamer and run the process.

In [ ]:
import torch
import sys
import os
import gc
from pathlib import Path
from src.inference_streaming import GEEInferenceStreamer

# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize Streamer
streamer = GEEInferenceStreamer(
    model_path=Path(MODEL_PATH),
    config_path=Path(CONFIG_PATH),
    device=device
)

# Run Inference
result = streamer.stream_inference(
    lat=LAT,
    lon=LON,
    year=YEAR,
    output_dir=full_output_path,
    project_id=PROJECT_ID
)

print("Inference Complete!")
print(result)

# Cleanup to free RAM
del streamer
gc.collect()
torch.cuda.empty_cache()

## 6. Visualize Result
Display the output CHM.

In [ ]:
import rasterio
from matplotlib import pyplot as plt
import numpy as np

output_path = result['path']

if result['status'] == 'success':
    with rasterio.open(output_path) as src:
        chm = src.read(1)
                
        plt.figure(figsize=(10, 10))
        plt.imshow(chm, cmap='viridis', vmin=0, vmax=3000) # Display up to 30m
        plt.colorbar(label='Canopy Height (cm)')
        plt.title(f"CHM Output for {YEAR} at {LAT}, {LON}")
        plt.axis('off')
        plt.show()